# ML Zoomcamp 2023 - Homework 10

## Downloading the model and code

In [1]:
!git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git

Cloning into 'machine-learning-zoomcamp'...
remote: Enumerating objects: 4875, done.
remote: Counting objects: 100% (2082/2082), done.
remote: Compressing objects: 100% (465/465), done.
remote: Total 4875 (delta 1774), reused 1631 (delta 1612), pack-reused 2793
Receiving objects: 100% (4875/4875), 10.97 MiB | 10.79 MiB/s, done.
Resolving deltas: 100% (3025/3025), done.


In [2]:
!mv machine-learning-zoomcamp/cohorts/2023/05-deployment/homework/* .

In [3]:
!rm -rf machine-learning-zoomcamp/

In [5]:
!docker build -t zoomcamp-model:hw10 .

[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.1s (2/3)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 293B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/svizor/zoomcamp-model:3.10.12-  0.1s
[+] Building 0.3s (2/3)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 293B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.

## Q1

After putting image to run in another terminal with ```docker run -it --rm -p 9696:9696 zoomcamp-model:hw10```

In [7]:
!python3 q6_test.py

{'get_credit': True, 'get_credit_probability': 0.726936946355423}


## Q2

In [8]:
!kind --version

kind version 0.20.0


## Q3

In [9]:
!kind create cluster

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.27.3) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Thanks for using kind! 😊


In [12]:
!kubectl cluster-info --context kind-kind

Kubernetes control plane is running at https://127.0.0.1:49647
CoreDNS is running at https://127.0.0.1:49647/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [13]:
!kubectl get service

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   6m44s


## Q4

In [14]:
!kind load docker-image zoomcamp-model:hw10

Image: "zoomcamp-model:hw10" with ID "sha256:08023598068a36ef47931f9ecc6254f215617dd09295f2ea578bf32a57993f97" not yet present on node "kind-control-plane", loading...


## Q5

In [15]:
!cat deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit
spec:
  selector:
    matchLabels:
      app: credit
  replicas: 1
  template:
    metadata:
      labels:
        app: credit
    spec:
      containers:
      - name: credit
        image: "zoomcamp-model:hw10"
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: "128Mi"
            cpu: "1"
        ports:
        - containerPort: 9696

In [16]:
!kubectl apply -f deployment.yaml

deployment.apps/credit created


In [17]:
!kubectl get pod

NAME                     READY   STATUS              RESTARTS   AGE
credit-9f67444b6-dpctx   0/1     ContainerCreating   0          0s


# Q6

In [18]:
!cat service.yaml

apiVersion: v1
kind: Service
metadata:
  name: credit
spec:
  type: LoadBalancer
  selector:
    app: credit
  ports:
  - port: 80
    targetPort: 9696

In [19]:
!kubectl apply -f service.yaml

service/credit created


In [20]:
!kubectl get service

NAME         TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
credit       LoadBalancer   10.96.221.162   <pending>     80:30966/TCP   5s
kubernetes   ClusterIP      10.96.0.1       <none>        443/TCP        7m42s


Ran ```kubectl port-forward service/credit 9696:80``` in another terminal

In [21]:
!python3 q6_test.py

{'get_credit': True, 'get_credit_probability': 0.726936946355423}
